<a href="https://colab.research.google.com/github/utsb-fmm/FHRMA/blob/master/FS%20training%20python%20sources/1_Prepare_packaged_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the dataset and prepare it in compact matrix by concatenating small recordings in constant width units:

Prepare datasets for FSDop, FSMHR, and FSScalp.

Associated paper in https://www.preprints.org/manuscript/202207.0131/v1

---
FHR Morphological Analysis Toolbox Copyright (C) 2022 Samuel Boudet, Faculté de Médecine et Maïeutique,
samuel.boudet@gmail.com

This file is part of FHR Morphological Analysis Toolbox

FHR Morphological Analysis Toolbox is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

 FHR Morphological Analysis Toolbox is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

In [1]:
import tensorflow as tf
import array
import numpy as np
import os
import time
import datetime
import pickle
from IPython.core.debugger import set_trace


In [2]:
#The two firsts lines are for Colab to write the packaged data in your google drive.
#Fill free to change for other platforms (but change also in other files)
from google.colab import drive
drive.mount('drive')
basefolder="drive/My Drive/FHRMA-Training-FS/"

!wget https://github.com/utsb-fmm/FHRMA/raw/master/FS%20training%20python%20sources/dataV8.zip
!unzip dataV8.zip
!cp DiffMF.dat 'drive/My Drive/FHRMA-Training-FS/'

Mounted at drive
--2022-07-20 18:57:21--  https://github.com/utsb-fmm/FHRMA/raw/master/FS%20training%20python%20sources/dataV8.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-20 18:57:21 ERROR 404: Not Found.

unzip:  cannot find or open dataV8.zip, dataV8.zip.zip or dataV8.zip.ZIP.


# I For FSMHR


In [10]:

nd=733
maxrow=200
#Package the MHR signals in units of length "samp", 
# the number of units will be minimized for training dataset and will be
#maxrowval for validation dataset
#maxrowval and number of unit in train dataset should be a multiple of your batch size
#so samp should be set to have the training dataset unit number is a multiple of batch size, and samp should be greater than the longest signal (to include it)
#We recommend setting the highest possible batch_size to lower computation time, but a too high batch_size will saturate GPU (or TPU) memory. 
#For TPU V3 and V2, batch_size should be a multiple of 8 (number of core TPU)
#Denpending of training computer and memory you should adjust those parameters

# For GPU
samp=57700 
maxrowval=13

u8=array.array("B")
Lfile=os.path.getsize("dataV8/isval.dat")
with open("dataV8/isval.dat", "rb") as binary_file:
    u8.fromfile(binary_file,Lfile) 
isval0=np.asarray(u8.tolist())>0

X0=();Y0=();L=();isval=();
for i in range(nd):
    Lt=os.path.getsize("dataV8/dop"+("%03d" % (i+1))+".dop")//10
    u16=array.array("H")
    with open("dataV8/dop"+("%03d" % (i+1))+".dop", "rb") as binary_file:
        u16.fromfile(binary_file,Lt*5)
    data=np.asarray(u16.tolist()).reshape(-1,5)/4

    isFalse=(data[:,2]<1)
    docare=(data[:,2]!=1)     
    isFalseM=(data[:,3]<1)
    docareM=(data[:,3]!=1)       

    if sum(docareM)>0: #+sum(docare) if FSDop and FSMHR train simultaneously
        k=np.sqrt(len(docareM)/(sum(docareM)))
        docareM=docareM*k       
    
        Ynp=np.concatenate((docare.reshape(1,-1,1),isFalse.reshape(1,-1,1),docareM.reshape(1,-1,1),isFalseM.reshape(1,-1,1)),axis=2)
        NotLoss=(data>0).astype(float)
        data=np.concatenate(((data[:,0:1]-120)/60*NotLoss[:,0:1],NotLoss[:,0:1],(data[:,1:2]-120)/60*NotLoss[:,1:2],NotLoss[:,1:2],data[:,4:5]),axis=1)
        Y0+=(Ynp,)
        X0+=(data,)   
        isval+=(isval0[i],)
        L+=(Lt,)

X0trainval=[()  for k in range(2)];Y0trainval=[() for k in range(2)];L0trainval=[()  for k in range(2)];
X0trainval[0]=[X0[k] for k in np.where(np.logical_not(isval))[0]]
Y0trainval[0]=[Y0[k] for k in np.where(np.logical_not(isval))[0]]
L0trainval[0]=np.asarray([L[k] for k in np.where(np.logical_not(isval))[0]])
X0trainval[1]=[X0[k] for k in np.where(isval)[0]]
Y0trainval[1]=[Y0[k] for k in np.where(isval)[0]]
L0trainval[1]=np.asarray([L[k] for k in np.where(isval)[0]])

nval=len(L0trainval[1])

X=[np.zeros((maxrow,samp,6)) for k in range(2)]
Y=[np.zeros((maxrow,samp,4)) for k in range(2)]

ListRec=[ [(),]*maxrow for k in range(2)]
LengthRec=[ [(),]*maxrow for k in range(2)]

if max(L0trainval[0])>samp:
    print("Warning: a training recording is not included")
    print(np.where(L0trainval[0]>samp))
# For train dataset, we fill the lines successyvely by adding the longer possible recording at the end of line until no recording fill
i=0
while min(L0trainval[0])<=samp:
    X[0][i,0,5]=1
    j=1
    while samp-j-1>=min(L0trainval[0]):
        k=np.argmax(L0trainval[0]*(L0trainval[0]<=samp-j-1))
        ListRec[0][i]+=(k,)
        LengthRec[0][i]+=(L0trainval[0][k],)
        X[0][i,j:j+L0trainval[0][k],0:5]=X0trainval[0][k]
        X[0][i,j+L0trainval[0][k],5]=1
        Y[0][i,j:j+L0trainval[0][k],0:4]=Y0trainval[0][k]
        j=j+L0trainval[0][k]+1
        L0trainval[0][k]=samp+1

    X[0][i,j:,5]=1
    i=i+1
X[0]=X[0][:i,:,:]
Y[0]=Y[0][:i,:,:]
ListRec[0]=ListRec[0][:i]
LengthRec[0]=LengthRec[0][:i]

# For validation dataset, we first put the longer recordings 
X[1]=X[1][:maxrowval,:,:]
Y[1]=Y[1][:maxrowval,:,:]
ListRec[1]=ListRec[1][:maxrowval]
LengthRec[1]=LengthRec[1][:maxrowval]    
CurrFill=np.ones(maxrowval,dtype=np.int32)
X[1][:,0,5]=1
while min(L0trainval[1])<=samp: #allocated rec hav L0 = samp+1
    k=np.argmax(L0trainval[1]*(L0trainval[1]<=samp)) #choose longer rec not allocated
    pos=np.argmin(CurrFill) # Search the less filled line
    #print("pos %d rec %d of length %d" % (pos,k,L0trainval[1][k]))
    ListRec[1][pos]+=(k,)
    LengthRec[1][pos]+=(L0trainval[1][k],)
    X[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:5]=X0trainval[1][k]
    X[1][pos,CurrFill[pos]+L0trainval[1][k],5]=1
    Y[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:4]=Y0trainval[1][k]
    CurrFill[pos]+=(L0trainval[1][k]+1)
    L0trainval[1][k]=samp+1

if np.any(L0trainval[1]!=samp+1):
    print("Warning: a test recording is not included")
    print(np.where(L0trainval[1]!=samp+1))

for i in range(maxrowval):
    X[1][i,CurrFill[i]:,5]=1

(X_train,Y_train,X_val,Y_val)=(X[0],Y[0],X[1],Y[1])
Y_val[:,:,2]=Y_val[:,:,2]/np.sum(Y_val[:,:,2])
Y_train[:,:,2]=Y_train[:,:,2]/np.sum(Y_train[:,:,2]) 

with open(basefolder+'dataV8MHR%d-%d.pkl'%(X_train.shape[0],X_val.shape[0]), 'wb') as f:  
    pickle.dump([X_train,Y_train,X_val,Y_val,ListRec,LengthRec], f)

X_train.shape

(13, 57700, 6)

# II For FSDop

In [11]:
nd=733
maxrow=200
#Package the Doppler signals in units of length "samp", 
# the number of units will be minimized for training dataset and will be
#maxrowval for validation dataset
#maxrowval and number of unit in train dataset should be a multiple of your batch size
#so samp should be set to have the training dataset unit number is a multiple of batch size, and samp should be greater than the longest signal (to include it)
#A too high batch_size will saturate GPU (or TPU) memory. We recommend setting the highest possible batch_size to lower computation time
#For TPU V3 and V2, batch_size should be a multiple of 8 (number of core TPU)
#Denpending of training computer and memory you should adjust those parameter

# For TPU V3
#samp=64800 # min for having 80 on train lines; batch_size 40
#maxrowval=40 

#For TPU V2 
#samp=72000 # min for having 72 on train lines; batch_size 24
#maxrowval=24

# For GPU
samp=64800 # min for having 80 on train lines; batch_size 20
maxrowval=20 

u8=array.array("B")
Lfile=os.path.getsize("dataV8/isval.dat")
with open("dataV8/isval.dat", "rb") as binary_file:
    u8.fromfile(binary_file,Lfile) 
isval0=np.asarray(u8.tolist())>0

X0=();Y0=();L=();isval=();
for i in range(nd):
    Lt=os.path.getsize("dataV8/dop"+("%03d" % (i+1))+".dop")//10
    u16=array.array("H")
    with open("dataV8/dop"+("%03d" % (i+1))+".dop", "rb") as binary_file:
        u16.fromfile(binary_file,Lt*5)
    data=np.asarray(u16.tolist()).reshape(-1,5)/4

    isFalse=(data[:,2]<1)
    docare=(data[:,2]!=1)     
    isFalseM=(data[:,3]<1)
    docareM=(data[:,3]!=1)       

    if sum(docare)>0:
        k=np.sqrt(len(docare)/(sum(docare)))
        docare=docare*k       

        Ynp=np.concatenate((docare.reshape(1,-1,1),isFalse.reshape(1,-1,1),docareM.reshape(1,-1,1),isFalseM.reshape(1,-1,1)),axis=2)
        NotLoss=(data>0).astype(float)
        data=np.concatenate(((data[:,0:1]-120)/60*NotLoss[:,0:1],NotLoss[:,0:1],(data[:,1:2]-120)/60*NotLoss[:,1:2],NotLoss[:,1:2],data[:,4:5]),axis=1)
        Y0+=(Ynp,)
        X0+=(data,)   
        isval+=(isval0[i],)
        L+=(Lt,)

X0trainval=[()  for k in range(2)];Y0trainval=[() for k in range(2)];L0trainval=[()  for k in range(2)];
X0trainval[0]=[X0[k] for k in np.where(np.logical_not(isval))[0]]
Y0trainval[0]=[Y0[k] for k in np.where(np.logical_not(isval))[0]]
L0trainval[0]=np.asarray([L[k] for k in np.where(np.logical_not(isval))[0]])
X0trainval[1]=[X0[k] for k in np.where(isval)[0]]
Y0trainval[1]=[Y0[k] for k in np.where(isval)[0]]
L0trainval[1]=np.asarray([L[k] for k in np.where(isval)[0]])

nval=len(L0trainval[1])

X=[np.zeros((maxrow,samp,6)) for k in range(2)]
Y=[np.zeros((maxrow,samp,4)) for k in range(2)]

ListRec=[ [(),]*maxrow for k in range(2)]
LengthRec=[ [(),]*maxrow for k in range(2)]

if max(L0trainval[0])>samp:
    print("Warning: a training recording is not included")
    print(np.where(L0trainval[0]>samp))
# For train dataset, we fill the lines successyvely by adding the longer possible recording at the end of line until no recording fill
i=0
while min(L0trainval[0])<=samp:
    X[0][i,0,5]=1
    j=1
    while samp-j-1>=min(L0trainval[0]):
        k=np.argmax(L0trainval[0]*(L0trainval[0]<=samp-j-1))
        ListRec[0][i]+=(k,)
        LengthRec[0][i]+=(L0trainval[0][k],)
        X[0][i,j:j+L0trainval[0][k],0:5]=X0trainval[0][k]
        X[0][i,j+L0trainval[0][k],5]=1
        Y[0][i,j:j+L0trainval[0][k],0:4]=Y0trainval[0][k]
        j=j+L0trainval[0][k]+1
        L0trainval[0][k]=samp+1

    X[0][i,j:,5]=1
    i=i+1
X[0]=X[0][:i,:,:]
Y[0]=Y[0][:i,:,:]
ListRec[0]=ListRec[0][:i]
LengthRec[0]=LengthRec[0][:i]

# For validation dataset, we first put the longer recordings 
X[1]=X[1][:maxrowval,:,:]
Y[1]=Y[1][:maxrowval,:,:]
ListRec[1]=ListRec[1][:maxrowval]
LengthRec[1]=LengthRec[1][:maxrowval]    
CurrFill=np.ones(maxrowval,dtype=np.int32)
X[1][:,0,5]=1
while min(L0trainval[1])<=samp: #allocated rec hav L0 = samp+1
    k=np.argmax(L0trainval[1]*(L0trainval[1]<=samp)) #choose longer rec not allocated
    pos=np.argmin(CurrFill) # Search the less filled line
    #print("pos %d rec %d of length %d" % (pos,k,L0trainval[1][k]))
    ListRec[1][pos]+=(k,)
    LengthRec[1][pos]+=(L0trainval[1][k],)
    X[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:5]=X0trainval[1][k]
    X[1][pos,CurrFill[pos]+L0trainval[1][k],5]=1
    Y[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:4]=Y0trainval[1][k]
    CurrFill[pos]+=(L0trainval[1][k]+1)
    L0trainval[1][k]=samp+1

if np.any(L0trainval[1]!=samp+1):
    print("Warning: a test recording is not included")
    print(np.where(L0trainval[1]!=samp+1))

for i in range(maxrowval):
    X[1][i,CurrFill[i]:,5]=1

(X_train,Y_train,X_val,Y_val)=(X[0],Y[0],X[1],Y[1])
Y_val[:,:,0]=Y_val[:,:,0]/np.sum(Y_val[:,:,0])
Y_train[:,:,0]=Y_train[:,:,0]/np.sum(Y_train[:,:,0]) 

with open(basefolder+'dataV8dop%d-%d.pkl'%(X_train.shape[0],X_val.shape[0]), 'wb') as f:  
    pickle.dump([X_train,Y_train,X_val,Y_val,ListRec,LengthRec], f)

X_train.shape

(80, 64800, 6)

# III For FSScalp

In [ ]:
nd=227
maxrow=200
#Package the Fetal Scalp ECG signals in units of length "samp",
# For TPU V3
#samp=30600 # min for having 40 on train lines
#maxrowval=40 

#For TPU V2 batch 32
#samp=38400
#maxrowval=32 

#For TPU V2 batch 24
#samp=51000 
#maxrowval=24 

#For TPU V2 btach 16
samp=38400 
maxrowval=16 

u8=array.array("B")
Lfile=os.path.getsize("dataV8/isvalint.dat")
with open("dataV8/isvalint.dat", "rb") as binary_file:
    u8.fromfile(binary_file,Lfile) 
isval0=np.asarray(u8.tolist())>0

X0=();Y0=();L=();isval=();
for i in range(nd):
    Lt=os.path.getsize("dataV8/int"+("%03d" % (i+1))+".int")//6
    u16=array.array("H")
    with open("dataV8/int"+("%03d" % (i+1))+".int", "rb") as binary_file:
        u16.fromfile(binary_file,Lt*3)
    data=np.asarray(u16.tolist()).reshape(-1,3)/4

    isFalse=(data[:,1]<1)
    docare=(data[:,1]!=1)           

    if sum(docare)>0:
        k=np.sqrt(len(docare)/(sum(docare)))
        docare=docare*k       

        Ynp=np.concatenate((docare.reshape(1,-1,1),isFalse.reshape(1,-1,1)),axis=2)
        NotLoss=(data>0).astype(float)
        data=np.concatenate(((data[:,0:1]-120)/60*NotLoss[:,0:1],NotLoss[:,0:1],data[:,2:3]),axis=1)
        Y0+=(Ynp,)
        X0+=(data,)   
        isval+=(isval0[i],)
        L+=(Lt,)

X0trainval=[()  for k in range(2)];Y0trainval=[() for k in range(2)];L0trainval=[()  for k in range(2)];
X0trainval[0]=[X0[k] for k in np.where(np.logical_not(isval))[0]]
Y0trainval[0]=[Y0[k] for k in np.where(np.logical_not(isval))[0]]
L0trainval[0]=np.asarray([L[k] for k in np.where(np.logical_not(isval))[0]])
X0trainval[1]=[X0[k] for k in np.where(isval)[0]]
Y0trainval[1]=[Y0[k] for k in np.where(isval)[0]]
L0trainval[1]=np.asarray([L[k] for k in np.where(isval)[0]])

nval=len(L0trainval[1])

X=[np.zeros((maxrow,samp,4)) for k in range(2)]
Y=[np.zeros((maxrow,samp,2)) for k in range(2)]

ListRec=[ [(),]*maxrow for k in range(2)]
LengthRec=[ [(),]*maxrow for k in range(2)]

# For train dataset, we fill the lines successyvely by adding the longer possible recording at the end of line until no recording fill
if max(L0trainval[0])>samp:
    print("Warning: a training recording is not included")
    print(np.where(L0trainval[0]>samp))
i=0
while min(L0trainval[0])<=samp:
    X[0][i,0,3]=1
    j=1
    while samp-j-1>=min(L0trainval[0]):
        k=np.argmax(L0trainval[0]*(L0trainval[0]<=samp-j-1))
        ListRec[0][i]+=(k,)
        LengthRec[0][i]+=(L0trainval[0][k],)
        X[0][i,j:j+L0trainval[0][k],0:3]=X0trainval[0][k]
        X[0][i,j+L0trainval[0][k],3]=1
        Y[0][i,j:j+L0trainval[0][k],0:2]=Y0trainval[0][k]
        j=j+L0trainval[0][k]+1
        L0trainval[0][k]=samp+1

    X[0][i,j:,3]=1
    i=i+1
X[0]=X[0][:i,:,:]
Y[0]=Y[0][:i,:,:]
ListRec[0]=ListRec[0][:i]
LengthRec[0]=LengthRec[0][:i]

# For validation dataset, we first put the longer recordings 
X[1]=X[1][:maxrowval,:,:]
Y[1]=Y[1][:maxrowval,:,:]
ListRec[1]=ListRec[1][:maxrowval]
LengthRec[1]=LengthRec[1][:maxrowval]    
CurrFill=np.ones(maxrowval,dtype=np.int32)
X[1][:,0,3]=1
while min(L0trainval[1])<=samp: #allocated rec hav L0 = samp+1
    k=np.argmax(L0trainval[1]*(L0trainval[1]<=samp)) #choose longer rec not allocated
    pos=np.argmin(CurrFill) # Search the less filled line
    #print("pos %d rec %d of length %d" % (pos,k,L0trainval[1][k]))
    ListRec[1][pos]+=(k,)
    LengthRec[1][pos]+=(L0trainval[1][k],)
    X[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:3]=X0trainval[1][k]
    X[1][pos,CurrFill[pos]+L0trainval[1][k],3]=1
    Y[1][pos,CurrFill[pos]:CurrFill[pos]+L0trainval[1][k],0:2]=Y0trainval[1][k]
    CurrFill[pos]+=(L0trainval[1][k]+1)
    L0trainval[1][k]=samp+1
if np.any(L0trainval[1]!=samp+1):
    print("Warning: a test recording is not included")
    print(np.where(L0trainval[1]!=samp+1))

for i in range(maxrowval):
    X[1][i,CurrFill[i]:,3]=1

(X_train,Y_train,X_val,Y_val)=(X[0],Y[0],X[1],Y[1])
Y_val[:,:,0]=Y_val[:,:,0]/np.sum(Y_val[:,:,0])
Y_train[:,:,0]=Y_train[:,:,0]/np.sum(Y_train[:,:,0]) 

with open(basefolder+'dataV8Int%d-%d.pkl'%(X_train.shape[0],X_val.shape[0]), 'wb') as f:  
    pickle.dump([X_train,Y_train,X_val,Y_val,ListRec,LengthRec], f)
    
X_train.shape

(32, 38400, 4)